In [42]:
# chapter 1
import numpy as np
import csv
import time
import sys
import time
np.random.seed(1234)

def randomize(): np.random.seed(time.time())

In [43]:
# chapter 1
# Hyper Parameter
RND_MEAN = 0     #정규분포 난숫값 평균
RND_STD = 0.0030     #정규분포 난숫값 표준편차
LEARNING_RATE = 0.001     #학습율

In [44]:
# chapter 1
# 실험용 메인 함수 
def NUM_exec(epoch_count=10, mb_size=10, report=1):
    global start
    start = time.time()
    load_NUM_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)

In [45]:
# 데이터 적재 함수 정의
# chapter3
def load_NUM_dataset():
    with open('./data/20x20_info.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)     # 첫행을 읽지 않고 건너뛴다
        rows = []
        for row in csvreader:
            rows.append(row)
            
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 400, 5  # 20x20 img dataset 형식에 맞춤 
    data = np.asarray(rows, dtype='float32')

In [47]:
# chapter1 (단층)
# 학습 및 평가함수
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)     # 데이터를 섞고 학습용/평가용 데이터셋 분리, 정렬 등
    global test_x, test_y , f, pm_output, start
    test_x , test_y = get_test_data()
    flag = 0
    for epoch in range(epoch_count):     # epoch만큼 학습 반복
        losses, accs = [], []
        #print(epoch)
        for n in range(step_count):     # 값 만큼 미니배치 처리
            train_x, train_y = get_train_data(mb_size, n) # train_x, train_y 는 
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
            
        if report > 0 and (epoch+1) % report == 0:  
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'. \
                  format(epoch+1, np.mean(losses), np.mean(accs), acc),file=f)
        if loss < 0.05 and flag != 1:
            flag = 1
            snapshot_epoch = epoch
            snapshot = time.time() - start
        # for_print_weight = np.reshape(pm_hidden['w'],(1,len(pm_hidden['w'])))
        # print(for_print_weight.tolist(),file=f_w)
        #wr_weight.writerow(for_print_weight.tolist())
    final_acc = run_test(test_x, test_y) # final_acc는 마지막 정확도만 존재
    print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc),file=f)
    if flag == 1 : print('Final Epoch : %s loss_zero time : Time : %s'%(snapshot_epoch+1,snapshot),file=f)
    else :
        snapshot = time.time() - start
        print('Final Epoch : %s Running Time : %s'%(epoch+1,snapshot),file=f)
    f.close()

In [1]:
# Chapter 1
# 학습 및 평가 데이터 획득 함수 정의
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0])     #인덱스 만들기
    np.random.shuffle(shuffle_map)     #인덱스 섞음
    step_count = int(data.shape[0] * 0.8) // mb_size    # 미니배치 학습에 필요한 배치 수
    test_begin_idx = step_count * mb_size    # train test 분류 경계 인덱스
    return step_count

def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    if nth == 0:     #epoch의 첫번째 호출에 학습데이터에 대한 부분적인 순서을 뒤섞음
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]     #미니배치 구간의 위치를 따져 그 구간에 해당하는 shuffle_map만 가져옴
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

In [49]:
# Chapter 1
# 학습 실행 함수와 평가 실행 함수 정의
def run_train(x, y):
    output, aux_nn = forward_neuralnet(x)     #순전파 처리
    loss, aux_pp = forward_postproc(output, y)     #순전파 처리
    accuracy = eval_accuracy(output, y)     #정확도 계산
    #aux_pp, aux_nn은 순전파 과정에서 확보 가능한 것들을 챙겨 역전파 함수에 전달
    #불필요한 계산을 줄여 효율을 높임
    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp)
    backprop_neuralnet(G_output, aux_nn)
    
    return loss, accuracy

def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

In [51]:
# chapter 3
# 후처리 과정에 대한 순전파와 역전파 함수의 재정의
# - 시그모이드 교차 엔트로피는 확률값 하나를 입력 받아 엔트로피 값을 계산
# - 소프트맥스 교차 엔트로피는 여러 확률값으로 구성된 확률 분포 하나로부터 엔트로피 값 구함
# - 미니배치 사이즈 N일때, 시그모이드 교차 엔트로피는 (N,1), 
# - 하지만 소프트맥스 교차 엔트로피는 (N,7) 부터 (N,1) 형태로 얻어짐
def forward_postproc(output, y):
    # output은 로짓값
    # 로짓값 -> 소프트맥스 교차 엔트로피를 -> loss 연산
    entropy = softmax_cross_entropy_with_logits(y, output)
    loss = np.mean(entropy) 
    return loss, [y, output, entropy]

def backprop_postproc(G_loss, aux):
    y, output, entropy = aux
    #평균(loss) 역산(편미분)
    g_loss_entropy = 1.0 / np.prod(entropy.shape)
    #엔트로피 함수 역산(편미분)
    g_entropy_output = softmax_cross_entropy_with_logits_derv(y, output)
    
    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy
    #손실 기울기
    return G_output

In [3]:
# chapter 3
# 정확도 함수 정의
def eval_accuracy(output, y):
    # np.argmax()는 각 로짓값을 담고 있는 최대값의 인덱스를 뽑아줌
    estimate = np.argmax(output, axis=1)
    answer = np.argmax(y, axis=1)
    # 둘 인덱스 간의 위치가 일치하면 True(1) 아니면 False(0)
    # wr.writerows(test_x)
    correct = np.equal(estimate, answer)
    return np.mean(correct)

In [53]:
# Chapter 3
# 소프트맥스 관련 함수 정의
def softmax(x):
    # 각 열에서 최대를 고르고(피쳐에 해당하는 최대)
    max_elem = np.max(x, axis=1)
    # 각 행을 전치시켜서 최댓값을 뺴주고 (행벡터 -> 열벡터)
    diff = (x.transpose() - max_elem).transpose()
    # 자연상수 처리 (열벡터)
    exp = np.exp(diff)
    # 자연상수 처리 한 것들을 합치고 (열벡터)
    sum_exp = np.sum(exp, axis=1)
    # 시그모이드 함수 수식 적용하고 전치 (열벡터 -> 행벡터)
    probs = (exp.transpose() / sum_exp).transpose()
    return probs

#실제로 호출되지 않지만 학습을 위한 구현
#소프트맥스 함수 자체보다 교차엔트로피 값을 중심으로 이루어지기 때문
def softmax_derv(x, y):
    mb_size, nom_size = x.shape
    derv = np.ndarray([mb_size, nom_size, nom_size])
    #3중 반복문을 통한 야코비안 행렬
    for n in range(mb_size):
        for i in range(nom_size):
            for j in range(nom_size):
                derv[n, i, j] = -y[n,i] * y[n,j]
            derv[n, i, i] += y[n,i]
    return derv

#log함수 폭주를 막기위해 epsilon 값을 10^(-10)값을 사용함
#labels는 원-핫 벡터
def softmax_cross_entropy_with_logits(labels, logits):
    probs = softmax(logits)
    return -np.sum(labels * np.log(probs+1.0e-10), axis=1)

def softmax_cross_entropy_with_logits_derv(labels, logits):
    return softmax(logits) - labels

In [54]:
# Chapter4
# 은닉 계층 하나를 위한 파라미터 생성 함수 정의
# - 단층 퍼셉트론에서 사용한 init_model() 함수와 비슷한 기능을 하지
# - 두 쌍의 파라미터를 저장하고 hidden_cnt라는 파라미터를 전달 받는 다는 점에서 다름
def init_model_hidden1():
    global pm_output, pm_hidden, input_cnt, output_cnt, hidden_cnt
    
    #은닉 계층 파라미터
    pm_hidden = alloc_param_pair([input_cnt, hidden_cnt])
    #출력 계층 파라미터
    pm_output = alloc_param_pair([hidden_cnt, output_cnt])
    
def alloc_param_pair(shape):
    #계층 하나를 위한 파라미터 쌍 생성
    weight = np.random.normal(RND_MEAN, RND_STD, shape)
    print(shape)
    #2차원 형태를 넘는 것을 대비해 편향은 shape의 마지막 순서로 할당
    bias = np.zeros(shape[-1])
    return {'w':weight, 'b':bias}

In [55]:
# Chapter 4
# 은닉 계층 하나를 위한 순전파 함수 정의
def forward_neuralnet_hidden1(x):
    # init_model_hidden1 쌍의 파라미터에 접근
    global pm_output, pm_hidden
    
    #입력 x와 pm_hidden을 통해 은닉계층 출력 계산 Relu함수 사용
    hidden = relu(np.matmul(x, pm_hidden['w']) + pm_hidden['b'])
    #hidden과 pm_output을 통해 출력계층 출력이자 최종출력 output 계산
    output = np.matmul(hidden, pm_output['w']) + pm_output['b']
    
    #출력 계층의 역전파 처리 때 가중치에 대한 편미분 정보로 hidden이 필요함
    return output, [x, hidden]

def relu(x):
    return np.maximum(x, 0)

In [56]:
# Chapter 4
# 은닉 계층 하나를 위한 역전파 함수 정의
def backprop_neuralnet_hidden1(G_output, aux):
    # G_output은 역전파 후처리 후만들어지는 파라미터
    global pm_output, pm_hidden
    
    # 순전파에 사용됐던 input
    x, hidden = aux
    
    # 출력층에 대한 역전파
    # 출력층에서 사용하는 input은 hidden 
    g_output_w_out = hidden.transpose()                      
    G_w_out = np.matmul(g_output_w_out, G_output)            
    G_b_out = np.sum(G_output, axis=0)    
    
    # 출력 계층과 은닉 계층 역전파 처리 매개하는 G_output으로부터 G_hidden을 구해내는 과정
    g_output_hidden = pm_output['w'].transpose()
    # pm_output['w']은 변환이 되는 계수이기 때문에 미리 기록한 것 위에 명시된 식에서 W_2
    G_hidden = np.matmul(G_output, g_output_hidden)   
    
    # 출력층 역전파
    pm_output['w'] -= LEARNING_RATE * G_w_out                
    pm_output['b'] -= LEARNING_RATE * G_b_out                
    
    # 은닉층 -> relu -> 출력층 순서
    # 역전파는 그 반대로 가야하기 때문에 Relu 부분을 처리해줘야 함 위 식에서 편미분값
    G_hidden = G_hidden * relu_derv(hidden)
    
    # 은닉층에 대한 역전파
    # 은닉층에서 사용하는 input은 x
    g_hidden_w_hid = x.transpose()
    G_w_hid = np.matmul(g_hidden_w_hid, G_hidden)            
    G_b_hid = np.sum(G_hidden, axis=0)                       
    
    # 은닉층 역전파
    pm_hidden['w'] -= LEARNING_RATE * G_w_hid                
    pm_hidden['b'] -= LEARNING_RATE * G_b_hid 
    
    
def relu_derv(y):
    return np.sign(y)

In [57]:
# Chapter4
# 가변적 은닉 계층 구성을 위한 파라미터 생성 함수 정의
def init_model_hiddens():
    # 은닉 계층의 수와 폭은 hidden_config리스트를 통해 지정
    global pm_output, pm_hiddens, input_cnt, output_cnt, hidden_config
    
    pm_hiddens = []
    prev_cnt = input_cnt
    
    # 리스트 성분 개수 = 계층의 수 | 리스트 성분 = 폭
    # 반복문을 통해 prev_cnt를 갱신해서 은닉층의 벡터 크기를 맞물리게 갱신
    for hidden_cnt in hidden_config:
        pm_hiddens.append(alloc_param_pair([prev_cnt, hidden_cnt]))
        prev_cnt = hidden_cnt
    
    #은닉층은 반복문이었지만 출력층은 하나이기 때문에 먼저 제시한 1개짜리 함수와 동일 
    pm_output = alloc_param_pair([prev_cnt, output_cnt])

In [58]:
# Chapter4
# 가변적 은닉 계층 구성을 위한 순전파 함수 정의
# - foward_neuralnet_hidden1 함수 를 은닉 계층 수만큼 반복해주면 됨
def forward_neuralnet_hiddens(x):
    # pm_hiddens는 앞에서 생성한 파라미터들
    global pm_output, pm_hiddens
    
    # 처음에 x로 input되어 첫번째 은닉층의 입력으로 이용됨
    hidden = x
    # input들이 정리된 리스트
    hiddens = [x]
    
    for pm_hidden in pm_hiddens:
        # hidden이 다음 층의 입력으로 초기화 됨
        hidden = relu(np.matmul(hidden, pm_hidden['w']) + pm_hidden['b'])
        hiddens.append(hidden)

    # 마지막에는 출력층의 입력으로 초기화
    output = np.matmul(hidden, pm_output['w']) + pm_output['b']
    
    # 출력층의 결과, 은닉층의 결과 리스트는 보조 정보로 활용
    return output, hiddens

In [59]:
#chapter 4
# 가변적 은닉 계층 구성을 위한 역전파 함수 정의
def backprop_neuralnet_hiddens(G_output, aux):
    global pm_output, pm_hiddens
    
    #foward_neuralnet_hiddens에서 파생된 hiddens가 aux로 input됨
    hiddens = aux
    
    # 출력층에서부터 역전파 시작 [-1]은 마지막 층을 출력을 가져오기 위해
    g_output_w_out = hiddens[-1].transpose()
    G_w_out = np.matmul(g_output_w_out, G_output)
    G_b_out = np.sum(G_output, axis=0)
    
    # 기울기 수정 전 Relu역전파를 위한 기울기 저장
    g_output_hidden = pm_output['w'].transpose() 
    G_hidden = np.matmul(G_output, g_output_hidden)
    
    # 출력층 기울기 수정
    pm_output['w'] -= LEARNING_RATE * G_w_out
    pm_output['b'] -= LEARNING_RATE * G_b_out
    
    # 반복문을 통해 은닉층 역전파, reversed()를 통해 뒤에서부터 가져옴
    # hiddens의 길이가 아닌 pm_hiddens의 길이를 반영
    # 처리해야하는 횟수는 은닉층의 개수 만큼 (pm_hiddens)이기 때문
    for n in reversed(range(len(pm_hiddens))):
        # Relu의 역전파 처리 부분
        # 입력벡터 pm_hiddens[n]를 relu함수의 역전파 처리를 위해 필요한 출력 벡터 내용에는 hiddens[n+1]로 접근
        G_hidden = G_hidden * relu_derv(hiddens[n+1])

        g_hidden_w_hid = hiddens[n].transpose()
        G_w_hid = np.matmul(g_hidden_w_hid, G_hidden)
        G_b_hid = np.sum(G_hidden, axis=0)
    
        g_hidden_hidden = pm_hiddens[n]['w'].transpose()
        G_hidden = np.matmul(G_hidden, g_hidden_hidden)

        pm_hiddens[n]['w'] -= LEARNING_RATE * G_w_hid
        pm_hiddens[n]['b'] -= LEARNING_RATE * G_b_hid

In [60]:
# Chapter 4
# 스위치 함수 정의
# - 전역 변수 설정에 따라 두 함수를 선택적으로 호출
global hidden_config, f

def init_model():
    # 값이 설정되어 있으면 
    if hidden_config is not None:
        print('은닉 계층 {}개를 갖는 다층 퍼셉트론이 작동되었습니다.'. \
              format(len(hidden_config)),file = f)
        init_model_hiddens()
        
    # 설정 되어있지 않으면
    else:
        print('은닉 계층 하나를 갖는 다층 퍼셉트론이 작동되었습니다.',file = f)
        init_model_hidden1()

# 신경망 순전파
def forward_neuralnet(x):
    if hidden_config is not None:
        return forward_neuralnet_hiddens(x)
    else:
        return forward_neuralnet_hidden1(x)

#신경망 역전파
def backprop_neuralnet(G_output, hiddens):
    if hidden_config is not None:
        backprop_neuralnet_hiddens(G_output, hiddens)
    else:
        backprop_neuralnet_hidden1(G_output, hiddens)

In [61]:
# chapter4
# 은닉 계층 구조 지정 함수 정의
def set_hidden(info):
    global hidden_cnt, hidden_config
    if isinstance(info, int):
        hidden_cnt = info
        hidden_config = None
    else:
        hidden_config = info

In [ ]:
def file_set(file_num):
    global f
    f = open('%s.txt'%file_num,'w')